# Check for Updates

**Current version : 2024.02.08**

In [ ]:
import ipywidgets as widgets
from IPython.utils import capture 
import requests

button = widgets.Button(description="Download now", button_style='success')
output = widgets.Output()

def on_button_clicked(b, url, version):
  with output:
    print('Downloading...')
    !wget -nv -O /notebooks/sdxl_webui_paperspace_{version}.ipynb {url}
    print(f'sdxl_webui_paperspace_{version}.ipynb downloaded in the root directory')

currentVersion = '2024.02.08'
updateURL = 'https://raw.githubusercontent.com/ffxvs/sd-webui-complete-setup/main/update.json'
response = requests.get(updateURL)

if response.status_code == 200:
    res = response.json()
    for data in res['data']:
        if data['id'] == 'paperspace':
            for variant in data['variants']:
                if variant['type'] == 'sdxl':
                    print('Current version : ' + currentVersion)
                    print('Latest version  : ' + variant['version'])
                    if (variant['version'] > currentVersion):
                        print('\nThere is new version')
                        button.on_click(lambda b: on_button_clicked(b, variant['url'], variant['version']))
                        display(button, output)
                        print('\nChangelog :')
                        print(variant['changelog'])
                    else:
                        print('\nThis is the latest version')                
else:
    print('Failed to check for updates') 
    print('Response code :', response.status_code)

# 1. Requirements

### 1.1. Function and Data
**REQUIRED EVERY TIME YOU WANT TO USE THIS NOTEBOOK**

In [ ]:
##################################### IMPORT #######################################

import os, re
import ipywidgets as widgets
from IPython.utils import capture
from IPython.display import clear_output

############################ ENVIRONMENT VARIABLES ################################

os.environ['LD_PRELOAD'] = '/lib/x86_64-linux-gnu/libtcmalloc_minimal.so.4'
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
os.environ['PYTHONWARNINGS'] = 'ignore'
os.environ['PIP_ROOT_USER_ACTION'] = 'ignore'
os.environ['PIP_DISABLE_PIP_VERSION_CHECK'] = '1'
os.environ['ACCELERATE'] = 'True'
os.environ['FORCE_CUDA'] = '1'
os.environ['ATTN_PRECISION'] = 'fp16'
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'garbage_collection_threshold:0.9,max_split_size_mb:512'
os.environ['CUDA_LAUNCH_BLOCKING'] = '0'
os.environ['CUDA_CACHE_DISABLE'] = '0'
os.environ['CUDA_AUTO_BOOST'] = '1'
os.environ['CUDA_MODULE_LOADING'] = 'LAZY'
os.environ['CUDA_DEVICE_DEFAULT_PERSISTING_L2_CACHE_PERCENTAGE_LIMIT'] = '0'
os.environ['GRADIO_ANALYTICS_ENABLED'] = 'False'
os.environ['SAFETENSORS_FAST_GPU'] = '1'
os.environ['NUMEXPR_MAX_THREADS'] = '16'

########################### GLOBAL PATHS AND FUNCTION ###########################

boolean = [False, True]

# Paths
root = '/notebooks'
a1111 = '/stable-diffusion-webui'
webui = root + '/stable-diffusion-webui-forge'
modulesPath = webui + '/modules'
outputsPath = webui + '/outputs'
extensionsPath = webui + '/extensions'
controlNetModelsPath = "models/ControlNet"
embeddingsPath = webui + "/embeddings"
modelsPath = webui + "/models/Stable-diffusion"
loraPath = webui + "/models/Lora"
upscalerPath = webui + "/models/ESRGAN"
vaePath = webui + "/models/VAE"
sharedStorage = root + '/shared-storage'
sharedModelsPath = sharedStorage + '/models'
sharedEmbeddingsPath = sharedStorage + '/embeddings'
sharedLoraPath = sharedStorage + '/lora'
sharedUpscalerPath = sharedStorage + '/esrgan'
sharedHypernetworksPath = sharedStorage + '/hypernetworks'
sharedVaePath = sharedStorage + '/vae'
sharedControlNetModelsPath = sharedStorage + '/controlNet'
sharedOutputsPath = sharedStorage + '/outputs'
sharedConfigPath = sharedStorage + '/config'
dependenciesPath = root + '/dependencies'
pythonLibsPath = '/root/.local/lib/python3.10'
packagesPath = pythonLibsPath + '/site-packages'


# Create symlink
def symlink(source, destination):
    if os.path.exists(source) and not os.path.islink(destination):
        !rm -r -f {destination}
        !ln -s {source} {destination}

# Complete message
def inf(msg, style, width): inf = widgets.Button(description=msg, disabled=True, button_style=style, layout=widgets.Layout(min_width=width));display(inf)

# Model Mapper
def modelMapper(name, version, url):
    return {
        'name': name,
        'version': version,
        'url': url
    }

# Selected Models
def selectedModels(models):
    modelList = []
    for model in models:
        isSelected = eval(model['id'])
        if isSelected != 'Select version...':
            for variant in model['variants']:
                if variant['version'] == isSelected:
                    modelList.append(modelMapper(
                        model['name'],
                        variant['version'],
                        variant['url']
                    ))
    return modelList

# Aria2c
def downloader(url, path, overwrite=False):
    args = '--download-result=hide --console-log-level=error -c -x 16 -s 16 -k 1M '
    if overwrite: args += '--allow-overwrite'
    if url.startswith('https://civitai.com/api/download/') and civitaiToken:
        if '?' in url:
            url = f'{url}&token={civitaiToken}'
        else:
            url = f'{url}?token={civitaiToken}'
    formattedURL = '"' + url + '"'
    if bool(re.search('\/[\w\.-]+\.\w+$', url)):
        filename = url.split('/')[-1]
        !aria2c {args} {formattedURL} -d {path} -o {filename}
    else:
        !aria2c {args} {formattedURL} -d {path}

# Git Clone
def silentClone(command, path, update=False, overwrite=False):
    directory = command.split('/')[-1]
    if os.path.exists(path + '/' + directory):
        if update:
            os.chdir(f'{path}/{directory}')
            !git pull -q
        elif overwrite:
            !rm -r {path}/{directory}
            !git clone -q --depth 10 {command} {path}/{directory}
    else:
        !git clone -q --depth 10 {command} {path}/{directory}

# WGet
def silentGet(command):
    !wget -nv {command}

# Resources downloader
def defaultResources(resourceList, resourcePath):
    for resource in resourceList:
        print('\n' + resource['name'] + '...')
        downloader(resource['url'], resourcePath)

def selectedResources(resourceList, resourcePath):
    for resource in resourceList:
        if eval(resource['id']):
            print('\n' + resource['name'] + '...')
            for url in resource['url']:
                downloader(url, resourcePath)
                print('')
                
def otherResources(resourceList, resourcePath):
    for resource in resourceList:
        print('\n' + resource)
        downloader(resource, resourcePath)

# Remove git from directory
def removeGit(directory):
    for (folder, subfolders, files) in os.walk(directory):
        if subfolders:
            for subfolder in subfolders:
                if subfolder == '.git':
                    pathFolder = os.path.join(folder, subfolder)
                    !rm -r {pathFolder}

        for file in files:
            if file == '.gitattributes' or file == 'readme.md' or file == 'README.md':
                pathFile = os.path.join(folder, file)
                !rm {pathFile}


################################ DATA OBJECT ################################
################################ CONTROLNET ################################

controlNetURLs = {
    'blur': ['https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/kohya_controllllite_xl_blur.safetensors'],
    'blur_anime': ['https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/kohya_controllllite_xl_blur_anime.safetensors'],
    'canny': ['https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/diffusers_xl_canny_mid.safetensors'],
    'canny_anime': ['https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/kohya_controllllite_xl_canny_anime.safetensors'],
    'depth': ['https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/diffusers_xl_depth_mid.safetensors'],
    'depth_anime': ['https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/kohya_controllllite_xl_depth_anime.safetensors'],
    'ipadapter': ['https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/ip-adapter_xl.pth'],
    'lineart': ['https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/t2i-adapter_diffusers_xl_lineart.safetensors'],
    'openpose': ['https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/thibaud_xl_openpose_256lora.safetensors'],
    'openpose_anime': ['https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/kohya_controllllite_xl_openpose_anime_v2.safetensors'],
    'recolor': ['https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/sai_xl_recolor_128lora.safetensors'],
    'scribble_anime': ['https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/kohya_controllllite_xl_scribble_anime.safetensors'],
    'sketch': ['https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/sai_xl_sketch_256lora.safetensors'],
    'softedge': ['https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/sargezt_xl_softedge.safetensors']
}

################################ EXTENSIONS ################################

defaultExtensions = [
    {
        'name': 'Aspect Ratio Helper',
        'url': 'https://github.com/thomasasfk/sd-webui-aspect-ratio-helper',
    },
    {
        'name': 'Canvas Zoom',
        'url': 'https://github.com/richrobber2/canvas-zoom',
    },
    {
        'name': 'Cleaner',
        'url': 'https://github.com/novitalabs/sd-webui-cleaner',
    },
    {
        'name': 'SD Delete Button',
        'url': 'https://github.com/reforget-id/sd_delete_button',
    },
    {
        'name': 'SD Model Downloader',
        'url': 'https://github.com/Iyashinouta/sd-model-downloader',
    },
    {
        'name': 'State',
        'url': 'https://github.com/ilian6806/stable-diffusion-webui-state',
    },
    {
        'name': 'Ultimate SD Upscale',
        'url': 'https://github.com/Coyote-A/ultimate-upscale-for-automatic1111',
    }
]

extensionList = [
    {
        'id': 'adetailer',
        'name': 'Adetailer (After Detailer)',
        'url': 'https://github.com/Bing-su/adetailer',
    },
    {
        'id': 'animateDiff',
        'name': 'AnimateDiff',
        'url': 'https://github.com/continue-revolution/sd-webui-animatediff',
    },
    {
        'id': 'bmab',
        'name': 'BMAB',
        'url': 'https://github.com/portu-sim/sd-webui-bmab'
    },
    {
        'id': 'depthLib',
        'name': 'Depth Map Library',
        'url': 'https://github.com/wywywywy/sd-webui-depth-lib',
    },
    {
        'id': 'huggingFace',
        'name': 'Hugging Face',
        'url': 'https://github.com/camenduru/stable-diffusion-webui-huggingface',
    },
    {
        'id': 'infiniteImg',
        'name': 'Infinite Image Browsing',
        'url': 'https://github.com/zanllp/sd-webui-infinite-image-browsing',
    },
    {
        'id': 'inpaintAny',
        'name': 'Inpaint Anything',
        'url': 'https://github.com/Uminosachi/sd-webui-inpaint-anything',
    },
    {
        'id': 'latentCouple',
        'name': 'Latent Couple',
        'url': 'https://github.com/aria1th/stable-diffusion-webui-two-shot',
    },
    {
        'id': 'miniPaint',
        'name': 'Mini Paint',
        'url': 'https://github.com/0Tick/a1111-mini-paint',
    },
    {
        'id': 'npw',
        'name': 'Negative Prompt Weight',
        'url': 'https://github.com/muerrilla/stable-diffusion-NPW',
    },
    {
        'id': 'openOutpaint',
        'name': 'openOutpaint',
        'url': 'https://github.com/zero01101/openOutpaint-webUI-extension',
    },
    {
        'id': 'photopea',
        'name': 'Photopea',
        'url': 'https://github.com/yankooliveira/sd-webui-photopea-embed',
    },
    {
        'id': 'promptHistory',
        'name': 'Prompt History',
        'url': 'https://github.com/namkazt/sd-webui-prompt-history',
    },
    {
        'id': 'rembg',
        'name': 'Remove Background',
        'url': 'https://github.com/AUTOMATIC1111/stable-diffusion-webui-rembg',
    },
    {
        'id': 'regionalPrompter',
        'name': 'Regional Prompter',
        'url': 'https://github.com/hako-mikan/sd-webui-regional-prompter',
    },
    {
        'id': 'sag',
        'name': 'Self Attention Guidance',
        'url': 'https://github.com/ashen-sensored/sd_webui_SAG',
    },
    {
        'id': 'styleXL',
        'name': 'Style Selector XL',
        'url': 'https://github.com/ahgsql/StyleSelectorXL'
    },
    {
        'id': 'tagComplete',
        'name': 'Tag Autocomplete',
        'url': 'https://github.com/DominikDoom/a1111-sd-webui-tagcomplete',
    }
]


#################################### MODELS #####################################
############################# ANIME / CARTOON /3D ###############################

animeModels = [
    {
        'id': 'animagine',
        'name': 'Animagine XL',
        'variants': [
            {
                'version': 'v3',
                'url': 'https://huggingface.co/cagliostrolab/animagine-xl-3.0/resolve/main/animagine-xl-3.0.safetensors'
            }
        ]
    },
    {
        'id': 'bluePencil',
        'name': 'Blue Pencil XL',
        'variants': [
            {
                'version': 'v4.0.1',
                'url': 'https://huggingface.co/bluepen5805/blue_pencil-XL/resolve/main/blue_pencil-XL-v4.0.1.safetensors'
            }
        ]
    },
    {
        'id': 'counterfeit',
        'name': 'CounterfeitXL',
        'variants': [
            {
                'version': 'v2.5',
                'url': 'https://huggingface.co/gsdf/CounterfeitXL-V2.0/resolve/main/CounterfeitXL-V2.5.safetensors'
            }
        ]
    },
    {
        'id': 'dynaVision',
        'name': 'DynaVision XL',
        'variants': [
            {
                'version': 'v0.6.1.0_vae',
                'url': 'https://civitai.com/api/download/models/297740?type=Model&format=SafeTensor&size=pruned&fp=fp16'
            }
        ]
    },
    {
        'id': 'sdxlNiji',
        'name': 'SDXL Niji',
        'variants': [
            {
                'version': 'SE',
                'url': 'https://huggingface.co/rafationgson/niji-sdxl/resolve/main/sdxlNijiSpecial_sdxlNijiSE.safetensors'
            }
        ]
    },
    {
        'id': 'unstableDiff',
        'name': 'SDXL Unstable Diffusers',
        'variants': [
            {
                'version': 'v11',
                'url': 'https://huggingface.co/frankjoshua/sdxlUnstableDiffusers_v11/resolve/main/sdxlUnstableDiffusers_v11.safetensors'
            },
            {
                'version': 'v11+RunDiffusion',
                'url': 'https://huggingface.co/Yamer-AI/SDXL_Unstable_Diffusers/resolve/main/Unstable%20Diffusers%20V11%20%2B%20RunDiffusion.safetensors'
            },
            {
                'version': 'Turbo v10',
                'url': 'https://civitai.com/api/download/models/247214?type=Model&format=SafeTensor&size=full&fp=fp16'
            }
        ]
    }
]

############################### GENERAL PURPOSE #################################

generalModels = [
    {
        'id': 'copaxTimeless',
        'name': 'Copax TimeLessXL',
        'variants': [
            {
                'version': 'v9',
                'url': 'https://civitai.com/api/download/models/293413?type=Model&format=SafeTensor&size=pruned&fp=fp16'
            },
            {
                'version': 'Turbo v1',
                'url': 'https://civitai.com/api/download/models/247259?type=Model&format=SafeTensor&size=pruned&fp=fp16'
            }
        ]
    },
    {
        'id': 'dreamShaper',
        'name': 'DreamShaper XL',
        'variants': [
            {
                'version': 'aplha2',
                'url': 'https://huggingface.co/Lykon/DreamShaper/resolve/main/DreamShaperXL1.0Alpha2_fixedVae_half_00001_.safetensors'
            },
            {
                'version': 'Turbo DPM++ SDE v2',
                'url': 'https://huggingface.co/Lykon/dreamshaper-xl-v2-turbo/resolve/main/DreamShaperXL_Turbo_v2.safetensors'
            }
        ]
    },
    {
        'id': 'juggernaut',
        'name': 'Juggernaut XL',
        'variants': [
            {
                'version': 'v8',
                'url': 'https://huggingface.co/Dremmar/juggernaut_v8/resolve/main/juggernautXL_v8Rundiffusion.safetensors'
            }
        ]
    },
    {
        'id': 'protoVision',
        'name': 'ProtoVision XL',
        'variants': [
            {
                'version': 'v6.6.0_vae',
                'url': 'https://civitai.com/api/download/models/265938?type=Model&format=SafeTensor&size=pruned&fp=fp16'
            }
        ]
    },
    {
        'id': 'sdxl',
        'name': 'SD XL',
        'variants': [
            {
                'version': 'v1_vae',
                'url': 'https://huggingface.co/stabilityai/stable-diffusion-xl-base-1.0/resolve/main/sd_xl_base_1.0_0.9vae.safetensors'
            },
            {
                'version': 'v1_vae-refiner',
                'url': 'https://huggingface.co/stabilityai/stable-diffusion-xl-refiner-1.0/resolve/main/sd_xl_refiner_1.0_0.9vae.safetensors'
            },
            {
                'version': 'Turbo v1',
                'url': 'https://huggingface.co/stabilityai/sdxl-turbo/resolve/main/sd_xl_turbo_1.0_fp16.safetensors'
            }
        ]
    }
]

################################## REALISTIC ####################################

realisticModels = [
    {
        'id': 'newReality',
        'name': 'NewRealityXL',
        'variants': [
            {
                'version': 'v2.1',
                'url': 'https://huggingface.co/nncyberpunk/NewRealityXL-2.1/resolve/main/newrealityxlAllInOne_21.safetensors'
            }
        ]
    },
    {
        'id': 'nightVision',
        'name': 'NightVision XL',
        'variants': [
            {
                'version': 'v0.7.9.1_vae',
                'url': 'https://civitai.com/api/download/models/247444?type=Model&format=SafeTensor&size=pruned&fp=fp16'
            }
        ]
    },
    {
        'id': 'realismEngine',
        'name': 'Realism Engine SDXL',
        'variants': [
            {
                'version': 'v3_vae',
                'url': 'https://huggingface.co/misri/realismEngineSDXL_v30VAE/resolve/main/realismEngineSDXL_v30VAE.safetensors'
            }
        ]
    },
    {
        'id': 'realEdge',
        'name': 'Realities Edge XL',
        'variants': [
            {
                'version': 'v5',
                'url': 'https://civitai.com/api/download/models/183894?type=Model&format=SafeTensor&size=pruned&fp=fp16'
            },
            {
                'version': 'Turbo v2',
                'url': 'https://civitai.com/api/download/models/294995?type=Model&format=SafeTensor&size=pruned&fp=fp16'
            }
        ]
    },
    {
        'id': 'realVis',
        'name': 'RealVisXL',
        'variants': [
            {
                'version': 'v3_vae',
                'url': 'https://huggingface.co/SG161222/RealVisXL_V3.0/resolve/main/RealVisXL_V3.0.safetensors'
            },
            {
                'version': 'v3_vae-inpainting',
                'url': 'https://civitai.com/api/download/models/297320?type=Model&format=SafeTensor&size=pruned&fp=fp16'
            },
            {
                'version': 'Turbo v3_vae',
                'url': 'https://huggingface.co/SG161222/RealVisXL_V3.0_Turbo/resolve/main/RealVisXL_V3.0_Turbo.safetensors'
            }
        ]
    }
]

#################################### WOMAN #####################################

womanModels = [
    {
        'id': 'arienmix',
        'name': 'ArienMixXL Asian portrait',
        'variants': [
          {
            'version': 'v4',
            'url': 'https://civitai.com/api/download/models/322265?type=Model&format=SafeTensor&size=pruned&fp=bf16'
          }
        ]
    },
    {
        'id': 'helloWorld',
        'name': 'LEOSAM\'s HelloWorld SDXL',
        'variants': [
            {
                'version': 'v5',
                'url': 'https://civitai.com/api/download/models/338512?type=Model&format=SafeTensor&size=full&fp=fp16'
            },
            {
                'version': 'Turbo+LCM v2',
                'url': 'https://civitai.com/api/download/models/243290?type=Model&format=SafeTensor&size=full&fp=fp16'
            }
        ]
    },
    
    {
        'id': 'sdvnReal',
        'name': 'SDVN6 RealXL',
        'variants': [
          {
            'version': 'Detailface',
            'url': 'https://civitai.com/api/download/models/134461?type=Model&format=SafeTensor&size=full&fp=fp16'
          }
        ]
    },
    {
        'id': 'xxmix9real',
        'name': 'XXMix 9realistic SDXL',
        'variants': [
          {
            'version': 'v1',
            'url': 'https://huggingface.co/Remilistrasza/XXMix_9realisticSDXL/resolve/main/xxmix9realisticsdxl_v10.safetensors'
          }
        ]
    }
]


########################################################################################################
############################################## LORA ####################################################

defaultLoraList = [
    {
        'name': 'Detail Tweaker XL',
        'url': 'https://huggingface.co/ffxvs/lora-effects-xl/resolve/main/detail_tweaker_xl.safetensors',
    }
]

loraList = [
    {
        'id': 'blurXL',
        'name': 'Blur SDXL',
        'url': ['https://huggingface.co/ffxvs/lora-effects-xl/resolve/main/blur_sdxl.safetensors'],
    },
    {
        'id': 'detailEyes',
        'name': 'DetailedEyes XL',
        'url': ['https://huggingface.co/ffxvs/lora-effects-xl/resolve/main/detailedEyes_v3.safetensors'],
    },
    {
        'id': 'lcmXL',
        'name': 'LCM Lora SDXL',
        'url': ['https://huggingface.co/ffxvs/lora-effects-xl/resolve/main/xl_more_art-full_v1.safetensors'],
    },
    {
        'id': 'lcmTurbo',
        'name': 'LCM&TurboMix',
        'url': [
            'https://huggingface.co/ffxvs/lora-effects-xl/resolve/main/LCMTurboMix_DPM_SDE_Karras.safetensors',
            'https://huggingface.co/ffxvs/lora-effects-xl/resolve/main/LCMTurboMix_Euler_A_fix.safetensors',
            'https://huggingface.co/ffxvs/lora-effects-xl/resolve/main/LCMTurboMix_LCM_Sampler.safetensors'
        ]
    },
    {
        'id': 'handsXL',
        'name': 'Hands XL',
        'url': ['https://huggingface.co/ffxvs/lora-effects-xl/resolve/main/hands_xl_v21.safetensors']
    },
    {
        'id': 'moreArt',
        'name': 'XL More Art Full',
        'url': ['https://huggingface.co/ffxvs/lora-effects-xl/resolve/main/xl_more_art-full_v1.safetensors']
    }
    
]


################################### EMBEDDINGS (TEXTUAL INVERSION) ####################################

defaultEmbeddingList = [
    {
        'name': 'Negative prompts pack XL',
        'url': 'https://huggingface.co/ffxvs/negative-prompts-pack-xl',
    },
    {
        'name': 'Embeddings Collection XL',
        'url': 'https://huggingface.co/ffxvs/embeddings-collection-xl',
    }
]

############################################## UPSCALER ##############################################

defaultUpscalerList = [
    {
        'name': 'UltraSharp',
        'url': 'https://huggingface.co/ffxvs/upscaler/resolve/main/UltraSharp_4x.pth',
    },
]

upscalerList = [
    {
        'id': 'bsrgan',
        'name': 'BSRGAN',
        'url': ['https://huggingface.co/ffxvs/upscaler/resolve/main/BSRGAN_4x.pth'],
    },
    {
        'id': 'lsdirPlus',
        'name': 'LSDIRPlus',
        'url': ['https://huggingface.co/ffxvs/upscaler/resolve/main/LSDIRplus_4x.pth'],
    },
    {
        'id': 'nomos8ksc',
        'name': 'Nomos8kSC',
        'url': ['https://huggingface.co/ffxvs/upscaler/resolve/main/Nomos8kSC_4x.pth'],
    },
    {
        'id': 'superscale',
        'name': 'NMKD Superscale',
        'url': ['https://huggingface.co/ffxvs/upscaler/resolve/main/NMKD_Superscale_4x.pth'],
    },
    {
        'id': 'remacri',
        'name': 'Remacri',
        'url': ['https://huggingface.co/ffxvs/upscaler/resolve/main/Remacri_4x.pth'],
    },
    {
        'id': 'ultraMix',
        'name': 'UltraMix',
        'url': [
            'https://huggingface.co/ffxvs/upscaler/resolve/main/UltraMix_Balanced_4x.pth',
            'https://huggingface.co/ffxvs/upscaler/resolve/main/UltraMix_Restore_4x.pth',
            'https://huggingface.co/ffxvs/upscaler/resolve/main/UltraMix_Smooth_4x.pth'
        ],
    },
    {
        'id': 'valar',
        'name': 'Valar',
        'url': ['https://huggingface.co/ffxvs/upscaler/resolve/main/Valar_4x.pth'],
    },
]


############################################## VAE ##############################################

defaultVaeList = [
    {
        'name': 'sdxl-vae-fp16-fix',
        'url': 'https://huggingface.co/ffxvs/vae-collection-xl/resolve/main/sdxl-vae-fp16-fix.safetensors',
    }
]

inf('\u2714 Completed','success', '30px')

### 1.2. Dependencies
**REQUIRED EVERY TIME YOU START THE MACHINE**

In [ ]:
print('⏳ Installing dependencies...')
os.chdir(root)

with capture.capture_output() as cap:
    !apt -y -q update
    !apt -y -q install aria2 git-lfs gcc g++ google-perftools
    !pip install -q torch==2.1.2 torchvision==0.16.2 torchaudio==2.1.2 --index-url https://download.pytorch.org/whl/cu121
    !pip install -q xformers==0.0.23.post1 ngrok onnxruntime-gpu insightface

print('\n')
inf('\u2714 Completed','success', '30px')

# 2. Setup Webui

### 2.1. Webui

In [ ]:
# Update Webui
updateWebui = boolean[0]


################################################################################################################

os.chdir(root)
print('⏳ Installing Stable Diffusion Webui Forge...')
silentClone('https://github.com/lllyasviel/stable-diffusion-webui-forge', root, updateWebui)
os.chdir(webui)

# Download configs
if not os.path.exists(f'{sharedConfigPath}/config.json'):
    downloader('https://raw.githubusercontent.com/ffxvs/sd-webui-complete-setup/main/configs/config.json', sharedConfigPath)
if not os.path.exists(f'{sharedConfigPath}/ui-config.json'):
    downloader('https://raw.githubusercontent.com/ffxvs/sd-webui-complete-setup/main/configs/ui-config.json', sharedConfigPath)

# Make symlinks from shared storage
symlink(sharedModelsPath, modelsPath)
symlink(sharedLoraPath, loraPath)
symlink(sharedEmbeddingsPath, embeddingsPath)
symlink(sharedUpscalerPath, upscalerPath)
symlink(sharedVaePath, vaePath)
symlink(sharedControlNetModelsPath, controlNetModelsPath)
symlink(sharedOutputsPath, outputsPath)
symlink(f'{sharedConfigPath}/config.json', f'{webui}/config.json')
symlink(f'{sharedConfigPath}/ui-config.json', f'{webui}/ui-config.json')

print('\n')
inf('\u2714 Completed','success', '30px')

### 2.2. Extensions

#### Extension List

**Pre-Installed Extensions :** 

* [Aspect Ratio Helper](https://github.com/thomasasfk/sd-webui-aspect-ratio-helper)
* [Canvas Zoom](https://github.com/richrobber2/canvas-zoom)
* [Cleaner](https://github.com/novitalabs/sd-webui-cleaner)
* [Controlnet](https://github.com/Mikubill/sd-webui-controlnet)
* [SD Delete Button](https://github.com/reforget-id/sd_delete_button)
* [SD Model Downloader](https://github.com/Iyashinouta/sd-model-downloader)
* [State](https://github.com/ilian6806/stable-diffusion-webui-state)
* [Ultimate SD Upscale](https://github.com/Coyote-A/ultimate-upscale-for-automatic1111)

In [ ]:
# UPDATE EXTENSIONS
updateExts = boolean[0]

# ControlNet - https://github.com/Mikubill/sd-webui-controlnet
allControlNet = boolean[0] # Download all controlNet models (The total size will be very large)

controlNet = {  # Select models to download 
                # Default models are : depth, ip-adapter, openpose
    'blur': boolean[0],
    'blur_anime': boolean[0],
    'canny': boolean[0],
    'canny_anime': boolean[0],
    'depth': boolean[0],
    'depth_anime': boolean[0],
    'ipadapter': boolean[1],
    'lineart': boolean[0],
    'openpose': boolean[0],
    'openpose_anime': boolean[0],
    'recolor': boolean[0],
    'scribble_anime': boolean[0],
    'sketch': boolean[0],
    'softedge': boolean[0],
}

# Adetailer (After Detailer) - https://github.com/Bing-su/adetailer
adetailer = boolean[1]

# AnimateDiff - https://github.com/continue-revolution/sd-webui-animatediff
animateDiff = boolean[0]

# BMAB - https://github.com/portu-sim/sd-webui-bmab
bmab = boolean[1]

# Depth Map Library - https://github.com/wywywywy/sd-webui-depth-lib
depthLib = boolean[0]

# Hugging Face - https://github.com/camenduru/stable-diffusion-webui-huggingface
huggingFace = boolean[0]

# Infinite Image Browsing - https://github.com/zanllp/sd-webui-infinite-image-browsing
infiniteImg = boolean[1]

# Inpaint Anything - https://github.com/Uminosachi/sd-webui-inpaint-anything
inpaintAny = boolean[0]

# Latent Couple - https://github.com/aria1th/stable-diffusion-webui-two-shot
latentCouple = boolean[0]

# Mini Paint - https://github.com/0Tick/a1111-mini-paint
miniPaint = boolean[1]

# Negative Prompt Weight - https://github.com/muerrilla/stable-diffusion-NPW
npw = boolean[0]

# openOutpaint - https://github.com/zero01101/openOutpaint-webUI-extension
openOutpaint = boolean[0]

# Photopea - https://github.com/yankooliveira/sd-webui-photopea-embed
photopea = boolean[1]

# Prompt History - https://github.com/namkazt/sd-webui-prompt-history
promptHistory = boolean[1]

# Regional Prompter - https://github.com/hako-mikan/sd-webui-regional-prompter
regionalPrompter = boolean[0]

# Remove Background - https://github.com/AUTOMATIC1111/stable-diffusion-webui-rembg
rembg = boolean[0]

# Self Attention Guidance - https://github.com/ashen-sensored/sd_webui_SAG
sag = boolean[0]

# Style Selector XL - https://github.com/ahgsql/StyleSelectorXL
styleXL = boolean[0]

# Tag Autocomplete - https://github.com/DominikDoom/a1111-sd-webui-tagcomplete
tagComplete = boolean[1]


################################################################################################################


print("⏳ Installing pre-installed extensions...")
for extension in defaultExtensions:
    print(extension['name'] + '...')
    silentClone(extension['url'], extensionsPath, updateExts)

print("\n⏳ Installing selected extensions...")
for extension in extensionList:
    if eval(extension['id']):
        print(extension['name'] + '...')
        silentClone(extension['url'], extensionsPath, updateExts)

if allControlNet:
    print("\n⏳ Download all acontrolNet models...")
    for model in controlNetURLs:
        print('\n' + model + '...')
        downloader(controlNetURLs[model][0], f'{controlNetModelsPath}/sdxl')
else:
    print("\n⏳ Download selected controlNet models...")
    for model in controlNet:
        if controlNet[model]:
            print('\n' + model + '...')
            downloader(controlNetURLs[model][0], f'{controlNetModelsPath}/sdxl')        

print('\n')        
%cd {webui}
print('\n')
inf('\u2714 Completed','success', '30px')

#### Install from URL

In [ ]:
# UPDATE EXTENSIONS
updateExts = boolean[0]

# Install extensions from URL
otherExtensions = []

if otherExtensions:
    print("⏳ Installing extensions...")  
    for extension in otherExtensions:
        name = extension.split('/')[-1]
        print(name + '...')
        silentClone(extension, extensionsPath, updateExts)

print('\n')        
%cd {webui}
print('\n')
inf('\u2714 Completed','success', '30px')

# 3. Launch Webui

Wait until `Model loaded in **.*s ...` appears.  
Then, click Gradio link `https://xxxxx.gradio.live` or Ngrok link `https://xxxxx.ngrok-free.app` to open WebUI.  
Always watch the output log to track image generation progress or errors. 

In [ ]:
boolean = [False, True]

# Dark theme
darkTheme = boolean[1]

# Optimizations
# For "opt-sdp-attention" and "opt-sdp-no-mem-attention" set directly from webui settings
xformers = boolean[1]

# Ngrok (Optional)
ngrokToken = ''
ngrokDomain = ''

# CORS (Optional) 
# separate with commas
cors = 'https://huchenlei.github.io'


################################################################################################################

import json 
%cd {webui}
print('⏳ Preparing...')
print('It will take a little longer for the first time...')
args = '--disable-safe-unpickle --enable-insecure-extension-access --gradio-queue --api --listen'

if darkTheme:
    args += ' --theme dark'
if xformers:
    args += ' --xformers'
if ngrokToken:
    args += ' --ngrok ' + ngrokToken
    if ngrokDomain:
        ngrokOptions = '\'{"hostname":"' + ngrokDomain + '", "request_header.add":"ngrok-skip-browser-warning:skip"}\''
        args += ' --ngrok-options ' + ngrokOptions
else:
    args += ' --share'
if cors:
    args += ' --cors-allow-origins ' + cors

accel = 'launch --num_cpu_threads_per_process=6'
with capture.capture_output() as cap:
    !accelerate {accel} launch.py --exit 
    !pip install -q pillow==9.5.0

print('Launching Webui...')
!accelerate {accel} webui.py {args}